In [1]:
import MeCab
import CaboCha
import pandas as pd
from tqdm import tqdm 
import regex

In [2]:
def get_word(tree, chunk):
    surface = ''
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞':
            surface += token.surface
        elif features[0] == '形容詞':
            surface += features[6]
            break
        elif features[0] == '動詞':
            surface += features[6]
            break
    return surface

def get_2_words(line):
    cp = CaboCha.Parser('-f1')
    tree = cp.parse(line)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1

    tuples = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface =  get_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface = get_word(tree, to_chunk)
            tuples.append((from_surface, to_surface))
    return tuples

In [3]:
#口コミを１文ずつに切り分けし、その文章内を係り受け解析し（単語A,単語B）にkawaii_wordがあればリストに加える
def taples_kawaii(txt):
    kawaii_word = ['可愛い','かわいい','カワイイ','美人','綺麗','きれい','美しい','美女']
    sen = regex.split(r'(?<=[。？])(?!$)', txt, flags=regex.VERSION1)
    add_list=[]
    for i in sen:
        tuples = get_2_words(i)
        for j in tuples:
            for k in kawaii_word:
                if k in j:
                    add_list.append(j)
    return add_list

In [4]:
#形態素解析するための関数を定義
tagger = MeCab.Tagger('-Owakati')#タグはMeCab.Tagger（neologd辞書）を使用したい
tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [5]:
#kawaii_wordが含まれた（単語A,単語B）をさらに形態素解析し、tennin_wordが含まれていたらフラグ立てをする
def tennin_hantei(i):

    txt = tokenize(i, 2, 10000, True)

    vector_list =[]

    a = 0

    tennin_word = ['店員','スタッフ','女将','女性','店主','助手','奥さん','奥様','店長']
    for k in txt:
        if k in tennin_word:
            a+=1
        else:
            a+=0        
    if a >=1:
        kawaii_flg = 1
    else:
        kawaii_flg = 0

    return kawaii_flg

In [6]:
df = pd.read_csv('processed_all_results.csv')
df.head()

,store_id,store_name,score,lunch_review,dinner_review
0,44,日本橋蛎殻町 すぎた,4.5,NaN,\n 2019年2月 再訪☆週末の夜、2回転目のすぎたさん。相変わらずのつ...
1,44,日本橋蛎殻町 すぎた,4.5,NaN,\n 【つまみ】相変わらず酒の進む佳肴の数々だ。◉皮剥 肝醤油◉長崎壱岐産...
2,44,日本橋蛎殻町 すぎた,5.0,NaN,\n この日は久々にかみさんと。「子供の面倒は？」という質問には黙秘権を使...
3,44,日本橋蛎殻町 すぎた,5.0,NaN,\n 今回訪問したのは日本最高の評価を受けるすぎた。予約困難店で、15日の...
4,44,日本橋蛎殻町 すぎた,4.6,NaN,\n 蛎殻町である。水天宮を通り抜け、到着。同行者が車のため、駐車場は近く...


In [7]:
pd.set_option("display.max_colwidth", 400)
list_sentence = df['dinner_review'].tolist()

new_list=[]
for txt in tqdm(list_sentence):
    new_list.append(taples_kawaii(txt))
kawaii_flg_list2=[]
kawaii_flg_list3=[]
for k in new_list:
    kawaii_flg_list=[]
    if len(k)==0:
        kawaii_flg_list.append(0)
    else: 
        for i in k:
                kawaii_flg_list.append(tennin_hantei(i))
                if tennin_hantei(i)==1:
                    kawaii_flg_list3.append(sum(kawaii_flg_list))

df['kawaii_flg']=kawaii_flg_list3
df_ramen = df.groupby(['store_name','score'])['dinner_review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)
df_ramen2 = df.groupby(['store_name','score'])['kawaii_flg'].sum().reset_index().sort_values('kawaii_flg', ascending=False)
df_ramen2

  0%|                                                                             | 15/17212 [00:10<2:34:03,  1.86it/s]

TypeError: expected string or buffer